In [2]:
from encode_data import read_from_file
from encode_data import generate_slices
from encode_data import generate_fourier_descriptions
from feature_generation.definitions import Atom, Point
from feature_generation.contour_descriptor import fourier_descriptor

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from feature_generation.alignment import align_catalyst
import matplotlib.pyplot as plt
from pyefd import reconstruct_contour
import numpy as np
from scipy.spatial.transform import Rotation as R

# Test rotational invariance

In [6]:
molecule = read_from_file("test_data/ir_tbp_1_dft-asme3_1_dft-asme3_1_dft-co_1_bromide_1_smi1_1_s_1_ts.xyz")
print(molecule)

rot3d = R.from_quat([0, 2, np.sin(np.pi/4), np.cos(np.pi/4)]).as_matrix()
molecule_rot = [Atom(ele, rot3d @ loc, rad) for (ele, loc, rad) in molecule]
print(molecule_rot)

[Atom(element='Ir', location=array([ 3.00000e-06, -3.60974e-01,  2.43672e-01]), radius=2.33), Atom(element='H', location=array([ 2.200000e-05, -3.126720e-01,  2.072583e+00]), radius=1.54), Atom(element='H', location=array([6.000000e-06, 5.450380e-01, 2.021628e+00]), radius=1.54), Atom(element='As', location=array([-2.418886, -0.174631,  0.037085]), radius=2.31), Atom(element='C', location=array([-2.976613,  0.462427, -1.722003]), radius=1.9), Atom(element='C', location=array([-3.256874,  1.106006,  1.25267 ]), radius=1.9), Atom(element='C', location=array([-3.490326, -1.79751 ,  0.276363]), radius=1.9), Atom(element='H', location=array([-2.427213,  1.405245, -1.908476]), radius=1.54), Atom(element='H', location=array([-4.071256,  0.627648, -1.754498]), radius=1.54), Atom(element='H', location=array([-2.677873, -0.288167, -2.477847]), radius=1.54), Atom(element='H', location=array([-2.730218,  2.069707,  1.112104]), radius=1.54), Atom(element='H', location=array([-3.115237,  0.759428,  

In [7]:
slices = generate_slices(molecule, 0.5, -5, 5, 0.01)
print(slices[9])

slices_rot = generate_slices(molecule_rot, 0.5, -5, 5, 0.01)
print(slices_rot[9])

[[3.03586692 2.42738231]
 [3.03579067 2.44263237]
 [3.03556193 2.45788089]
 ...
 [3.03553737 2.39568004]
 [3.03577816 2.41092838]
 [3.03586645 2.42617837]]
[[-0.2591793  -2.04771334]
 [-0.25925294 -2.03298637]
 [-0.25947384 -2.01826087]
 ...
 [-0.25950283 -2.07858106]
 [-0.25926777 -2.06385578]
 [-0.25917998 -2.04912889]]


In [8]:
def q(x):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.scatter(*zip(*list(slices[x])))
    ax1.set_aspect('equal')

    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(*zip(*list(slices_rot[x])))
    ax2.set_aspect('equal')
    plt.show()

interact(q, x=widgets.IntSlider(min=1, max=len(slices), step=1, value=9));

interactive(children=(IntSlider(value=9, description='x', max=20, min=1), Output()), _dom_classes=('widget-int…

In [19]:
fourier = generate_fourier_descriptions(slices, 20)
print(fourier)

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+

In [20]:
def q(x):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.scatter(*zip(*list(slices[x])))
    ax1.set_aspect('equal')
    contour = reconstruct_contour(fourier[x])
    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(*zip(*list(contour)))
    ax2.set_aspect('equal')
    plt.show()

interact(q, x=widgets.IntSlider(min=1, max=len(fourier-1), step=1, value=9));

interactive(children=(IntSlider(value=9, description='x', max=20, min=1), Output()), _dom_classes=('widget-int…